# Hourly Timeseries Simulation

While the static load flow cases are useful, there is a need within the industry for more depth in order to understand how new generators or network reinforcements impact the system during normal operation. Existing optimized economic dispatch tools such as PLEXOS and GAMS are used within power system planning to simulate realistic operation while meeting various constraints. These tools are useful within the industry, but remain computationally intensive, expensive, and take signficiant time to produce outputs with different sensitivites such as ECP constraint reports. 

In light of the recent decision on Large Energy User connection, there will be a push for Eirgrid to create a network capacity tool similar to what has been created for the distribution network by ESB. It may not be feasible to have a new transmission network capacity tool supported by data from quarterly simulations using PLEXOS. Instead, this pandapower based tool is a capable alternative. 

# Method

The first stage in the timeseries simulation part of this tool consists of creating a balanced power system model for each hour. The model is given a set of hourly values representing demand factor of peak, wind capacity factor and solar capacity factor. Each hour seeks to maximize the amount of solar and wind generation dispatch with respect to demand until convergence is reached. In doing so, a general dispatch priority order is utilized rather than a detailed economic dispatch. For example, if wind and solar generation only fills 10% of demand, battery storage is dispatched next, followed by interconnectors, followed by gas generators. Likewise, if wind and solar generation is high, battery storage will import, interconnectors can export, and conventional generators will be set to their minimum levels. 

This leaves room for edits, but keeps the focus on hourly load flow and realistic system dispatch. If the user would like specific battery storage projects with duration of 8 hours and others with 4 hour duration, they are able to incorporate that. Currently the script operates dispatching all battery storage uniformly, charging when renewable generation is high and discharging when renewables are low. This is another area that can be modified, if specific projects seek to dispatch based on local generation/demand rather than central system-wide dispatch. Likewise, edits can be made for specifc large energy users even though the currently operation assumes uniform demand behavior according to the system wide hourly demand factor. 

A balanced and converged network model is created for each hour at a rate of roughly 1463 hours (~2 months) in under 25 minutes using standard AC newton raphson powerflow solutions.

After the dispatch balancing is completed for each network model, the nodal analysis begins. Contingency analysis is completed for each hour, identifying the worst case contingency for each branch in the system greater or equal to 110kV. For each branch that experiences a worst case contingency greater than a user-defined "overload threshold", shift factors are gathered and aggregated according to each bus. In doing this, the user can determine which nodes have a greater contribution to all worst case overloads for each hour in the simulation. 

![ECP GSS](Dungarvan_area.png)